Factual CoCo Implementation





In [ ]:
import torch

# Set random seed for reproducibility
torch.manual_seed(42)

In [ ]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
import spacy

class Tokenizer(object):
    def __init__(self, model='en_core_web_sm'):
        super(Tokenizer, self).__init__()
        self.nlp_model = spacy.load(model)

    def tokenize(self, text):
        result = self.nlp_model(text)
        tokens = [x.text for x in result]
        return tokens

def get_coco_score(model, source_doc, generated_summ):
    inputs = tokenizer(source_doc, return_tensors="pt", truncation=True, padding=True)
    summ_inputs = tokenizer(generated_summ, return_tensors="pt", truncation=True, padding=True)
    summ_result = model(**inputs)
    summ_score = summ_result.logits.softmax(dim=1).cpu().detach().numpy()[:, 1]
    return summ_score[0]

if __name__ == '__main__':
    # Load BERT model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer_spacy = Tokenizer()

    # Finetune
    source_sentence = "I am Prianka . I am 22 years old. Last few days I have been suffering from insomnia. This never happened to me before. I want to take medications for Insomnia.Please help me to get my sleep back"
    summary_sentence = "What are the medications for Insomnia"
    # summary_sentence = "I have insomnia.Is there medicine for this?"

    # Tokenize source and summary sentences
    source_tokens = tokenizer_spacy.tokenize(source_sentence)
    summary_tokens = tokenizer_spacy.tokenize(summary_sentence)

    # Convert tokens to strings
    source_text = ' '.join(source_tokens)
    summary_text = ' '.join(summary_tokens)

    # Get COCO score
    coco_score = get_coco_score(model, source_text, summary_text)
    print("COCO Score:", coco_score)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


COCO Score: 0.49146122


In [ ]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
import spacy

class Tokenizer(object):
    def __init__(self, model='en_core_web_sm'):
        super(Tokenizer, self).__init__()
        self.nlp_model = spacy.load(model)

    def tokenize(self, text):
        result = self.nlp_model(text)
        tokens = [x.text for x in result]
        return tokens

def get_coco_score(model, tokenizer, source_doc, generated_summ):
    inputs = tokenizer(source_doc, return_tensors="pt", truncation=True, padding=True)
    summ_inputs = tokenizer(generated_summ, return_tensors="pt", truncation=True, padding=True)
    summ_result = model(**inputs)
    summ_score = summ_result.logits.softmax(dim=1).cpu().detach().numpy()[:, 1]
    return summ_score[0]

if __name__ == '__main__':
    # Load BERT model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer_spacy = Tokenizer()

    # Finetune
    source_sentence = "I am Prianka . I am 22 years old. Last few days I have been suffering from insomnia. This never happened to me before. I want to take medications for Insomnia.Please help me to get my sleep back"
    summary_sentence = "What are the medications for Insomnia"

    # Tokenize source and summary sentences
    source_tokens = tokenizer_spacy.tokenize(source_sentence)
    summary_tokens = tokenizer_spacy.tokenize(summary_sentence)

    # Convert tokens to strings
    source_text = ' '.join(source_tokens)
    summary_text = ' '.join(summary_tokens)

    # Get COCO score
    coco_score = get_coco_score(model, tokenizer, source_text, summary_text)
    print("COCO Score:", coco_score)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


COCO Score: 0.2701149


AlignScore Implementation

In [ ]:
pip install torch==1.12.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 888.4 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.12.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.12.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.12.1 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 1.12.1 which is incompatible.


In [ ]:
!git clone https://github.com/yuh-zha/AlignScore.git

Cloning into 'AlignScore'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 202 (delta 45), reused 36 (delta 18), pack-reused 131
Receiving objects: 100% (202/202), 531.19 KiB | 9.49 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [ ]:
%cd /content/AlignScore
!pip install .

/content/AlignScore
Processing /content/AlignScore
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 21.1 MB/s eta 0:00:00
  Created wheel for alignscore: filename=alignscore-0.1.3-py3-none-any.whl size=18368 sha256=75c924d1803107777c3d5c48e701b6963574531d382b0b8d22c7b2fbf0c8e6

In [ ]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
pip install -r /content/AlignScore/requirements.txt

  Cloning https://github.com/google-research/bleurt (to revision cebe7e6f996b40910cfaa520a63db47807e3bf5c) to /tmp/pip-install-co70vqod/bleurt_70ae097675764c3a8ec4d033a2dc1313
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt /tmp/pip-install-co70vqod/bleurt_70ae097675764c3a8ec4d033a2dc1313
  Running command git rev-parse -q --verify 'sha^cebe7e6f996b40910cfaa520a63db47807e3bf5c'
  Running command git fetch -q https://github.com/google-research/bleurt cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Resolved https://github.com/google-research/bleurt to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.1 MB/s et

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from alignscore import AlignScore

scorer = AlignScore(model='roberta-base', batch_size=32, device='cpu' , ckpt_path='/content/drive/MyDrive/model checkpoints/AlignScore-base.ckpt', evaluation_mode='nli_sp')
# score = scorer.score(contexts=['I have been suffering from insomnia for a couple of days. Its really disturbing. I want medications for this.Please help me get my sleep back'], claims=['What are the medicines for insomnia?']) # 0.26232191920280457
score = scorer.score(contexts=['I have been suffering from insomnia for a couple of days. Its really disturbing. I want medications for this.Please help me get my sleep back'], claims=['Insomnia is popular among teenagers.']) # 0.0012368602911010385


print(score)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file drive/MyDrive/model checkpoints/AlignScore-base.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in t

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Evaluating: 100%|██████████| 1/1 [00:04<00:00,  4.21s/it]

[0.0012368602911010385]


In [ ]:
import pandas as pd
import numpy as np
from alignscore import AlignScore

df = pd.read_csv('/content/zero-shot-summaries-1.csv')
scorer = AlignScore(model='roberta-base', batch_size=32, device='cpu', ckpt_path='/content/drive/MyDrive/model checkpoints/AlignScore-base.ckpt', evaluation_mode='nli_sp')

scores = []
for index, row in df.iterrows():
    score = scorer.score(contexts=[row['Ground_Truth']], claims=[row['Predicted_Summary']])
    print(score)
    scores.append(score)


mean_score = np.mean(scores)
print(f"Mean Accuracy Score: {mean_score}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file drive/MyDrive/model checkpoints/AlignScore-base.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in t

[0.17177417874336243]


Evaluating: 100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


[0.8950148224830627]


Evaluating: 100%|██████████| 1/1 [00:02<00:00,  2.48s/it]


...

[0.8996636867523193]


Evaluating: 100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


[0.06737276166677475]


Evaluating: 100%|██████████| 1/1 [00:02<00:00,  2.49s/it]

[0.04936755448579788]
Mean Accuracy Score: 0.2186883631156915


In [ ]:
import pandas as pd
import numpy as np
from alignscore import AlignScore

df = pd.read_csv('/content/finetuned-summaries-1.csv')
scorer = AlignScore(model='roberta-base', batch_size=32, device='cpu', ckpt_path='/content/drive/MyDrive/model checkpoints/AlignScore-base.ckpt', evaluation_mode='nli_sp')

scores = []
for index, row in df.iterrows():
    score = scorer.score(contexts=[row['Ground_Truth']], claims=[row['Predicted_Summary']])
    scores.append(score)


mean_score = np.mean(scores)
print(f"Mean Accuracy Score: {mean_score}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file drive/MyDrive/model checkpoints/AlignScore-base.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in t

Mean Accuracy Score: 0.48477933171205223
